In [284]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_file, ColumnDataSource
output_notebook()

# lib
import sys
sys.path.append('../')
from metrics import compute_metrics, _compute_metrics
from analysis import load_backfill, plot_accuracy, plot_cases, plot_prediction_interval, plot_metric_for_dates

Loading BokehJS ...

In [306]:
from datetime import timedelta
ground_truth = pd.read_csv('../data/austria/data.csv', index_col='region').transpose()
ground_truth.index = pd.to_datetime(ground_truth.index)
ground_truth.index.set_names(["date"], inplace=True)
ground_truth.tail()

region,Amstetten,Baden,Bludenz,Braunau am Inn,Bregenz,Bruck an der Leitha,Bruck-Mürzzuschlag,Deutschlandsberg,Dornbirn,Eferding,...,Waidhofen an der Ybbs(Stadt),Weiz,Wels(Stadt),Wels-Land,Wien(Stadt),Wiener Neustadt(Land),Wiener Neustadt(Stadt),Wolfsberg,Zell am See,Zwettl
date,,,,,,,,,,,,,,,,,,,,,
2020-05-27,301.0,172.0,257.0,132.0,274.0,108.0,52.0,50.0,153.0,53.0,...,25.0,197.0,58.0,79.0,3120.0,86.0,56.0,44.0,360.0,73.0
2020-05-28,301.0,173.0,257.0,132.0,275.0,110.0,52.0,50.0,153.0,53.0,...,25.0,197.0,58.0,79.0,3147.0,86.0,56.0,44.0,360.0,73.0
2020-05-29,301.0,173.0,257.0,132.0,275.0,112.0,52.0,50.0,153.0,53.0,...,25.0,198.0,58.0,79.0,3170.0,86.0,56.0,44.0,360.0,73.0
2020-05-30,301.0,173.0,257.0,132.0,275.0,112.0,52.0,50.0,153.0,53.0,...,25.0,198.0,58.0,79.0,3215.0,86.0,56.0,44.0,360.0,73.0
2020-05-31,301.0,173.0,257.0,132.0,275.0,112.0,52.0,50.0,153.0,53.0,...,25.0,198.0,58.0,79.0,3229.0,86.0,56.0,44.0,360.0,73.0


In [312]:
date = '20200524'
forecast = pd.read_csv(f'/private/home/maxn/covid19_spread/forecasts/austria/forecast-at-{date}.csv', index_col='date', parse_dates=['date'])
_compute_metrics(ground_truth, forecast, 0)

,2020-05-25 00:00:00,2020-05-26 00:00:00,2020-05-27 00:00:00,2020-05-28 00:00:00,2020-05-29 00:00:00,2020-05-30 00:00:00,2020-05-31 00:00:00
Measure,,,,,,,
RMSE,2.505549,2.579126,4.576046,4.481769,4.714904,2.749450,3.540829
MAE,0.484064,0.611282,0.952522,1.010206,1.074437,0.910188,1.046770
MAPE,0.002340,0.003726,0.004958,0.005502,0.005934,0.006416,0.006868
RMSE_NAIVE,1.414214,0.930580,1.366009,1.112256,1.414214,3.788207,3.533346
MAE_NAIVE,0.268041,0.226804,0.422680,0.391753,0.515464,0.845361,0.835052
MAE_MASE,1.805933,2.695198,2.253527,2.578683,2.084408,1.076686,1.253539
RMSE_MASE,1.771691,2.771525,3.349939,4.029440,3.333940,0.725792,1.002118


In [345]:
job = '2020_05_03_13_54/5788732_19'
job = '2020_05_03_17_54/5793635_19'
job = '2020_05_03_19_03/5795326_3'
job = '2020_05_03_19_12/5796126_28'
job = '2020_05_03_22_06/5800610_29'
job = '2020_05_03_22_28/5800710_29'
job = '2020_05_10_13_58/5820378_6'
job = '2020_05_24_19_21/5961397_3'
#job = '2020_05_24_19_45/5961415_3'
job = "2020_05_31_19_39/6000504_4"
jobdir = '/checkpoint/maxn/covid19/forecasts/at'
df_val = pd.read_csv(f'{jobdir}/{job}/validation-at.csv', index_col='date')
df_forecast = pd.read_csv(f'{jobdir}/{job}/../forecasts/forecast_best_mae.csv', index_col='date')
df_gt = ground_truth.loc[pd.to_datetime(df_mean.index)]

df_forecast['Wien(Stadt)'] += (np.arange(7))
df_forecast.to_csv('/private/home/maxn/covid19_spread/forecasts/austria/forecast-at-20200531.csv')
display(df_forecast['Wien(Stadt)'].to_frame())

n = len(ground_truth)
def plot_prediction_interval(county, p, start=0, color=('red', 'blue')):
    y = df_val[county].values
    df = pd.DataFrame({
        'x': range(start + 1, n + 1),
        'y': ground_truth[county].values[start:]
    })
    source = ColumnDataSource(df)
    p.line(x='x', y='y', source=source, color=color[0], line_width=2, line_dash='dotted')
    
    x = range(n + 1 - len(y), n + 1)
    df = pd.DataFrame({'x': x , 'y': df_val[county]})
    source = ColumnDataSource(df)
    #p.line(x='x', y='y', source=source, line_width=2, color=color[1])
    x = range(n, n + len(df_forecast))
    df = pd.DataFrame({'x': x , 'y': df_forecast[county]})
    source = ColumnDataSource(df)
    p.line(x='x', y='y', source=source, line_width=2, color=color[2])
    p.output_backend = 'svg'
    return p

p = figure(title=f"Prediction Austria, {len(df_val)} days", plot_width=500, plot_height=400, tools='save,hover')
for i, region in enumerate(regions):
    color = int(np.ceil(i + 20 / len(regions) * 256))
    p = plot_prediction_interval(region, p, 0, ('DarkOrange', 'SkyBlue', 'black'))
show(p)

,Wien(Stadt)
date,
2020-06-01,3249.891411
2020-06-02,3271.352638
2020-06-03,3292.861261
2020-06-04,3311.328827
2020-06-05,3332.441370
2020-06-06,3354.499443
2020-06-07,3374.343317


In [331]:
diff_gt = ground_truth.iloc[-1] - ground_truth.iloc[-8]
diff_fc = df_forecast.iloc[-1]  - df_forecast.iloc[0]
df_comp = pd.DataFrame([(diff_fc - diff_gt).abs(), diff_gt, diff_fc]).transpose().round(2)
df_comp.columns = ['Diff', 'GT', "FC"]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_comp.sort_values(by='GT'))

,Diff,GT,FC
Amstetten,0.38,0.0,0.38
Sankt Veit an der Glan,0.25,0.0,0.25
Sankt Pölten(Stadt),0.80,0.0,0.80
Sankt Pölten(Land),0.10,0.0,0.10
Sankt Johann im Pongau,0.03,0.0,0.03
Salzburg(Stadt),0.07,0.0,0.07
Rust(Stadt),0.00,0.0,0.00
Rohrbach,0.21,0.0,0.21
Ried im Innkreis,0.07,0.0,0.07
Scheibbs,0.04,0.0,0.04


In [343]:
daily_inc = ground_truth['Wien(Stadt)'].diff().to_frame()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(daily_inc[-7:])

,Wien(Stadt)
date,
2020-05-25,6.0
2020-05-26,29.0
2020-05-27,7.0
2020-05-28,27.0
2020-05-29,23.0
2020-05-30,45.0
2020-05-31,14.0


In [277]:
for i in range(len(daily_inc) - 1, 0, -7):
    print(daily_inc.index[i-6], daily_inc.index[i], daily_inc.iloc[i-6:i+1].sum())

2020-05-25 2020-05-31 Wien(Stadt)    151.0
dtype: float64
2020-05-18 2020-05-24 Wien(Stadt)    173.0
dtype: float64
2020-05-11 2020-05-17 Wien(Stadt)    233.0
dtype: float64
2020-05-04 2020-05-10 Wien(Stadt)    134.0
dtype: float64
2020-04-27 2020-05-03 Wien(Stadt)    181.0
dtype: float64
2020-04-20 2020-04-26 Wien(Stadt)    159.0
dtype: float64
2020-04-13 2020-04-19 Wien(Stadt)    198.0
dtype: float64
2020-04-06 2020-04-12 Wien(Stadt)    299.0
dtype: float64
2020-03-30 2020-04-05 Wien(Stadt)    600.0
dtype: float64
2020-03-23 2020-03-29 Wien(Stadt)    658.0
dtype: float64
2020-05-27 2020-03-22 Wien(Stadt)    0.0
dtype: float64


In [299]:
region = 'Wien(Stadt)'
region = 'Bruck an der Leitha'
region = 'Mödling'
region = 'Baden'

mets = []
for region in ground_truth.columns:
    _met = _compute_metrics(ground_truth[region].to_frame(), forecast[region].to_frame())
    _met = _met.iloc[:, -1].to_frame()
    _met.columns = [region]
    mets.append(_met)
mets = pd.concat(mets, axis=1).transpose()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(mets)

Measure,RMSE,MAE,MAPE,RMSE_NAIVE,MAE_NAIVE,MAE_MASE,RMSE_MASE
Amstetten,1.359536,1.359536,0.004517,0.0,0.0,inf,inf
Baden,8.361929,8.361929,0.048335,1.0,1.0,8.361929,8.361929
Bludenz,0.141958,0.141958,0.000552,1.0,1.0,0.141958,0.141958
Braunau am Inn,0.536906,0.536906,0.004067,0.0,0.0,inf,inf
Bregenz,1.670761,1.670761,0.006075,1.0,1.0,1.670761,1.670761
Bruck an der Leitha,0.326714,0.326714,0.002917,7.0,7.0,0.046673,0.046673
Bruck-Mürzzuschlag,0.041486,0.041486,0.000798,0.0,0.0,inf,inf
Deutschlandsberg,0.564381,0.564381,0.011288,1.0,1.0,0.564381,0.564381
Dornbirn,0.351766,0.351766,0.002299,0.0,0.0,inf,inf
Eferding,0.031647,0.031647,0.000597,0.0,0.0,inf,inf
